In [1]:
import keras
import datetime
import tensorflow                       as tf
from tensorflow.keras.callbacks         import TensorBoard
from tensorflow.keras.layers            import Input, Lambda,UpSampling2D, Conv2D,Dropout,MaxPooling2D,Conv2DTranspose,concatenate,BatchNormalization, Activation
from tensorflow.keras.models            import Model
from tensorflow.keras.optimizers        import Adam,RMSprop
from keras.utils                        import plot_model
from tensorflow.keras                   import layers, models
from tensorflow.keras.losses            import mae
import sys
import os
import numpy as np
import math
import random, time
from pathlib                        import Path
from PIL                            import Image

import skimage                      as ski
from   skimage.filters              import threshold_otsu
from   skimage                      import io, color
from   skimage.color                import rgb2gray
from   skimage                      import filters
import cv2                          as cv
import matplotlib.pyplot            as plt 
import gc
import glob
from skimage                        import img_as_ubyte
from skimage                        import io
import shutil
tf.keras.backend.clear_session()

2025-05-20 13:12:46.152820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747761166.214507    9357 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747761166.232473    9357 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747761166.362177    9357 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747761166.362196    9357 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747761166.362199    9357 computation_placer.cc:177] computation placer alr

### <font color="red">1. Processing data</font>


#### <font color="red">1.1. Selecting a Sample</font>

In [2]:
general_path_data = '/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5'

subdirectories = ['/Geometry','/Magnitude','/Pression','/U001','/U002']

geo_path =  general_path_data + subdirectories[0]
v_path   =  general_path_data + subdirectories[1]
p_path   =  general_path_data + subdirectories[2]
vx_path  =  general_path_data + subdirectories[3]
vy_path  =  general_path_data + subdirectories[4]

In [57]:
#method for reading image    
def get_img(img_name):
    return ski.io.imread(img_name)

def resizing_img(x,new_width=200):
   
    height=x.shape[0]
    width=x.shape[1]
    ratio = height / width
    new_height = int(new_width * ratio)
    y =cv.resize(x,(new_width,new_height))
    return y

# method for turning to a grey or binary image 
def processing(image,option):
        x = get_img(image)  
        x = rgb2gray(x)               # It returns a grayscale image with floating point values in the range from 0 to 1
        y = resizing_img(x) # Reshape image 
    
        # Binary option otherwise gray
        if (option):
            y=ski.util.img_as_ubyte(y)  # Convert it back to the original data type and the data range back 0 to 255. 
                                        # It is often better to use image values represented by floating point values 
            best_value_threshold=np.round(filters.threshold_otsu(y)) #  Otsu’s method calculates an “optimal” threshold

            _,y= cv.threshold(y, best_value_threshold, 255, cv.THRESH_BINARY)
            y=y/255.
        y=y[:, :, np.newaxis]
        return y

def create_file(path,name='Images_masked'):
     newpath = path +'/' +name
     os.makedirs(newpath, exist_ok=True)
     return newpath

In [26]:
#newpath = '/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Pression/p_00020.png'
'/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked'
'/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/npy_pression'

200


In [58]:
def create_npy(path_origin,name_sub_file,option,n_sample = 1000):
    images = sorted([os.path.join(path_origin,fname) for fname in os.listdir(path_origin) if fname.endswith(".png")])
    current_path = os.getcwd() 
    if (n_sample <= 20000):

        I = images[1:n_sample+1]
        newpath1=create_file(current_path)
        new_root = newpath1 +'/' +name_sub_file
        os.makedirs(new_root, exist_ok=True)
   
        for img in I:
            name_img = os.path.splitext(os.path.basename(img))[0]
            save_in=new_root+'/'+ name_img+'.npy'
            print(save_in)
            x=processing(img,option)
            np.save(save_in,x)  

In [ ]:
def mask(path1,path2,save_in):
    path_1 = Path(path1)
    file_npy_1=sorted(list(path_1.glob('*.npy')))
    
    path_2 = Path(path2)
    file_npy_2=sorted(list(path_2.glob('*.npy')))

    for f1, f2 in zip(file_npy_1,file_npy_2):
            array_1 = np.load(f1)
            array_2 = np.load(f2)
            array_masked=array_1*array_2
            np.save(save_in,array_masked)
    

In [ ]:
def create_file_masked():
    path =  os.getcwd() 
    path= create_file(path,name='Final_mask'):
    

In [ ]:
# Mask each image
def mask (name_file,name_data,path_file_1,path_file_2=None):
    print('enter')
    if path_file_2 == None:
        path_1 = Path(path_file_1)
        file_npy_1=sorted(list(path_1.glob('*.npy')))
        current_directory = Path.cwd()
        save_in = str(current_directory) + '/' + name_file 
        os.makedirs(save_in, exist_ok=True)
        n = 1
        for f1 in file_npy_1:
            array_1 = np.load(f1)
            save= save_in+ '/'+name_data+f'_{n:05d}.png'
            array_2d = img_as_ubyte(array_1.squeeze())
            io.imsave(save, array_2d)
            n=n+1
    else:
        path_1 = Path(path_file_1)
        path_2 = Path(path_file_2)
        file_npy_1=sorted(list(path_1.glob('*.npy')))
        file_npy_2=sorted(list(path_2.glob('*.npy')))
        current_directory = Path.cwd()
        save_in = str(current_directory) + '/' + name_file 
        os.makedirs(save_in, exist_ok=True)
        n = 1 
        for f1, f2 in zip(file_npy_1,file_npy_2):
            array_1 = np.load(f1)
            array_2 = np.load(f2)
            array_masked=array_1*array_2
            save= save_in+ '/'+name_data+f'_{n:05d}.png'
            array_2d = img_as_ubyte(array_masked.squeeze())
            io.imsave(save, array_2d)
            n=n+1

In [60]:
create_npy(p_path,'prueba2',False,n_sample = 10)

p_00001
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/prueba2/p_00001.npy
p_00002
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/prueba2/p_00002.npy
p_00003
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/prueba2/p_00003.npy
p_00004
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/prueba2/p_00004.npy
p_00005
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/prueba2/p_00005.npy
p_00006
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/prueba2/p_00006.npy
p_00007
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/prueba2/p_00007.npy
p_00008
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/prueba2/p_00008.npy
p_00009
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/prueba2/p_00009.npy
p_00010
/home/ppgi/Trabajo/predicting-flow-patterns/Images_masked/prueba2/p_00010.npy


In [ ]:
create_npy(p_path,'prueba',False,n_sample = 5)